In [1]:
!pip install transformers==4.44.2 joblib==1.4.2 scikit-learn==1.6.0 numpy==1.26.4 pandas==2.2.3 scipy==1.13.1 seaborn==0.13.2 tqdm==4.66.5 lightgbm==4.5.0 xgboost==2.1.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 72.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.8/301.8 kB 21.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.5/13.5 MB 80.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 46.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.4/78.4 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.9/153.9 MB 10.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 82.2 MB/s eta 0:00:00:00:01
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.67.1
    

In [2]:
import os
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader, random_split
from sklearn.model_selection import train_test_split
from torch.optim import AdamW
from transformers import AutoTokenizer, AutoModelForSequenceClassification

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [3]:
# Load datasets
train_df = pd.read_csv('/kaggle/input/rrck-set/Train_RRCK.csv')
train_df = train_df[['ID', 'SMILES', 'Permeability']]
test_df = pd.read_csv('/kaggle/input/rrck-set/Test_RRCK.csv')
test_df = test_df[['ID', 'SMILES', 'Permeability']]

In [19]:
tokenizer = AutoTokenizer.from_pretrained('seyonec/ChemBERTa-zinc-base-v1')
model = AutoModelForSequenceClassification.from_pretrained('seyonec/ChemBERTa-zinc-base-v1', num_labels=1)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Custom dataset class
class SMILESDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_length=325):
        self.tokenizer = tokenizer
        self.dataframe = dataframe
        self.max_length = max_length

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        smiles = self.dataframe.iloc[idx]['SMILES']
        permeability = self.dataframe.iloc[idx]['Permeability']
        inputs = self.tokenizer(smiles, return_tensors='pt', padding="max_length", truncation=True, max_length=self.max_length)
        
        input_ids = inputs['input_ids'].squeeze(0)  # Shape: (sequence_length,)
        attention_mask = inputs['attention_mask'].squeeze(0)  # Shape: (sequence_length,)
        
        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'labels': torch.tensor(permeability, dtype=torch.float)
        }


# datasets
train_dataset = SMILESDataset(train_df, tokenizer)
test_dataset = SMILESDataset(test_df, tokenizer)
batch_size = 16
# data loaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at seyonec/ChemBERTa-zinc-base-v1 and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [20]:
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)
num_epochs = 20

In [21]:
# Training loop
from tqdm import tqdm
for epoch in range(num_epochs):
    print(f"Entered Epoch {epoch + 1}")
    model.train()
    train_loss = 0

    for batch in tqdm(train_loader, desc=f'Training Epoch {epoch + 1}/{num_epochs}', unit='batch'):
        optimizer.zero_grad()

        # Move all batch tensors to device
        batch = {k: v.to(device) for k, v in batch.items()}
        labels = batch["labels"].unsqueeze(1)  # still shape: (batch_size, 1)

        # Forward pass
        outputs = model(
            input_ids=batch["input_ids"],
            attention_mask=batch["attention_mask"],
            labels=labels
        )
        loss = outputs.loss
        train_loss += loss.item()

        # Backprop and optimizer step
        loss.backward()
        optimizer.step()

    avg_train_loss = train_loss / len(train_loader)
    print(f'Epoch {epoch + 1}/{num_epochs} - Train Loss: {avg_train_loss:.4f}')

Entered Epoch 1


Training Epoch 1/20: 100%|██████████| 9/9 [00:04<00:00,  2.20batch/s]


Epoch 1/20 - Train Loss: 7.9182
Entered Epoch 2


Training Epoch 2/20: 100%|██████████| 9/9 [00:04<00:00,  2.08batch/s]


Epoch 2/20 - Train Loss: 0.6787
Entered Epoch 3


...

Epoch 18/20 - Train Loss: 0.1270
Entered Epoch 19


Training Epoch 19/20: 100%|██████████| 9/9 [00:04<00:00,  2.13batch/s]


Epoch 19/20 - Train Loss: 0.1312
Entered Epoch 20


Training Epoch 20/20: 100%|██████████| 9/9 [00:04<00:00,  2.13batch/s]

Epoch 20/20 - Train Loss: 0.1283


In [22]:
model_name = 'ChemBERTa_model_1_rrck'
model_save_path = f'/kaggle/working/{model_name}'
os.makedirs(model_save_path, exist_ok=True)

tokenizer.save_pretrained(model_save_path)
model.save_pretrained(model_save_path)

print(f'Model and tokenizer saved to {model_save_path}')

Model and tokenizer saved to /kaggle/working/ChemBERTa_model_1_rrck


In [23]:
from scipy.stats import pearsonr, spearmanr

model.eval()
test_loss = 0
test_true_labels = []
predictions = []

with torch.no_grad():
    for batch in tqdm(test_loader, desc='Testing', unit='batch'):
      
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].unsqueeze(1).to(device).float()

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        test_loss += loss.item()

        test_true_labels.extend(labels.cpu().numpy())
        preds = outputs.logits.squeeze().cpu().numpy()  
        predictions.extend(preds)

# Final test loss
avg_test_loss = test_loss / len(test_loader)
print(f'Test Loss: {avg_test_loss:.4f}')

test_true_labels = np.array(test_true_labels).flatten()
predictions = np.array(predictions)
print(test_true_labels.shape)
print(predictions.shape)

# Performance metrics
mse = mean_squared_error(test_true_labels, predictions)
rmse = np.sqrt(mse)
mae = mean_absolute_error(test_true_labels, predictions)
r2 = r2_score(test_true_labels, predictions)
PCC,_ = pearsonr(test_true_labels, predictions)
SCC,_ = spearmanr(test_true_labels, predictions)
# Print performance metrics
print(f'Mean Squared Error: {mse:.4f}')
print(f'Root Mean Squared Error: {rmse:.4f}')
print(f'Mean Absolute Error: {mae:.4f}')
print(f'R^2 Score: {r2:.4f}')
print(f'Pearson Correlation Coefficient: {PCC:.4f}')
print(f'Spearman Correlation Coefficient: {SCC:.4f}')

# Print hyperparameters
print("Hyperparameters:")
print(f"Learning Rate: {5e-5}")
print(f"Batch Size: 16")
print(f"Epochs: {num_epochs}")

Testing: 100%|██████████| 3/3 [00:00<00:00,  7.70batch/s]

Test Loss: 0.5346
(36,)
(36,)
Mean Squared Error: 0.6779
Root Mean Squared Error: 0.8233
Mean Absolute Error: 0.5847
R^2 Score: 0.0622
Pearson Correlation Coefficient: 0.4044
Spearman Correlation Coefficient: 0.4998
Hyperparameters:
Learning Rate: 5e-05
Batch Size: 16
Epochs: 20


In [24]:
import os
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from torch.optim import AdamW
from transformers import AutoTokenizer, AutoModel

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

model_name = 'ChemBERTa_model_1_rrck'
model_save_path = f'/kaggle/working/{model_name}'

if not os.path.exists(model_save_path):
    raise FileNotFoundError(f"The model directory {model_save_path} does not exist.")

tokenizer = AutoTokenizer.from_pretrained(model_save_path)
model = AutoModel.from_pretrained(model_save_path).to(device)

Some weights of RobertaModel were not initialized from the model checkpoint at /kaggle/working/ChemBERTa_model_1_rrck and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [25]:
# Load your datasets
train_df = pd.read_csv('/kaggle/input/rrck-set/Train_RRCK.csv')
train_df = train_df[['ID', 'SMILES', 'Permeability']]
test_df = pd.read_csv('/kaggle/input/rrck-set/Test_RRCK.csv')
test_df = test_df[['ID', 'SMILES', 'Permeability']]

In [26]:
train_encodings = tokenizer(list(train_df['SMILES']), truncation=True, padding=True, max_length=325, return_tensors="pt")
test_encodings = tokenizer(list(test_df['SMILES']), truncation=True, padding=True, max_length=325, return_tensors="pt")

In [27]:
from tqdm import tqdm 
batch_size = 16 

def generate_embeddings(encodings, batch_size):
    embeddings = []
    model.eval() 
    with torch.no_grad():
        for i in tqdm(range(0, len(encodings['input_ids']), batch_size), desc="Processing batches"):
            batch = {key: val[i:i + batch_size].to(device) for key, val in encodings.items()}  
            outputs = model(**batch)
            embeddings.append(outputs.last_hidden_state)
    return torch.cat(embeddings, dim=0)

In [28]:
train_embeddings = generate_embeddings(train_encodings, batch_size)
print(train_embeddings.shape)
train_embeddings = torch.mean(train_embeddings, dim=1)
print(train_embeddings.shape)

Processing batches: 100%|██████████| 9/9 [00:00<00:00, 11.52it/s]

torch.Size([140, 188, 768])
torch.Size([140, 768])


In [29]:
column_names = [f'x_fine_emb_ChemBerta{i}' for i in range(train_embeddings.shape[1])]
embeddings_df = pd.DataFrame(data=train_embeddings.cpu().numpy(), columns=column_names)
train_data = pd.concat([train_df, embeddings_df], axis=1)

In [30]:
test_embeddings = generate_embeddings(test_encodings, batch_size)
print(test_embeddings.shape)
test_embeddings = torch.mean(test_embeddings, dim=1)
print(test_embeddings.shape)

Processing batches: 100%|██████████| 3/3 [00:00<00:00, 15.16it/s]

torch.Size([36, 190, 768])
torch.Size([36, 768])


In [31]:
column_names = [f'x_fine_emb_ChemBerta{i}' for i in range(test_embeddings.shape[1])]
embeddings_df = pd.DataFrame(data=test_embeddings.cpu().numpy(), columns=column_names)
test_data = pd.concat([test_df, embeddings_df], axis=1)

In [33]:
train_data.to_csv("/kaggle/working/Train_ChemBERTa-zinc-base-v1_fine_tuned_embeddings_rrck.csv",index=False)
test_data.to_csv("/kaggle/working/Test_ChemBERTa-zinc-base-v1_fine_tuned_embeddings_rrck.csv",index=False)

In [34]:
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor, ExtraTreesRegressor
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
import xgboost as xgb
import lightgbm as lgb
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from scipy.stats import pearsonr, spearmanr
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [35]:
train_data = pd.read_csv("/kaggle/working/Train_ChemBERTa-zinc-base-v1_fine_tuned_embeddings_rrck.csv")
test_data = pd.read_csv("/kaggle/working/Test_ChemBERTa-zinc-base-v1_fine_tuned_embeddings_rrck.csv")

In [36]:
def train_and_test_predict(models, X_train, y_train, X_test, y_test):
    kf = KFold(n_splits=5, shuffle=True, random_state=101)
    results = {}
    predictions = []  

    for model in models:
        model_name = model.__class__.__name__
        predictions_train = []
        actual_y_train = []

        test_predictions_folds = []

        

        for train_index, val_index in kf.split(X_train):
            X_train_fold, X_val_fold = X_train.iloc[train_index], X_train.iloc[val_index]
            y_train_fold, y_val_fold = y_train.iloc[train_index], y_train.iloc[val_index]

            model.fit(X_train_fold, y_train_fold)

            y_pred_fold = model.predict(X_val_fold)
            y_pred_fold = np.clip(y_pred_fold, -10, -3.9)
            predictions_train.extend(y_pred_fold)
            actual_y_train.extend(y_val_fold)

            predictions_test_fold = model.predict(X_test)
            predictions_test_fold = np.clip(predictions_test_fold, -10, -3.9)
            test_predictions_folds.append(predictions_test_fold)


        mse_train = mean_squared_error(actual_y_train, predictions_train)
        mae_train = mean_absolute_error(actual_y_train, predictions_train)
        rmse_train = np.sqrt(mse_train)
        r2_train = r2_score(actual_y_train, predictions_train)
        pearson_train, _ = pearsonr(actual_y_train, predictions_train)
        spearman_train, _ = spearmanr(actual_y_train, predictions_train)


        predictions_test_mean = np.mean(test_predictions_folds, axis=0)
        predictions_test_std = np.std(test_predictions_folds, axis=0)

        mse_test = mean_squared_error(y_test, predictions_test_mean)
        mae_test = mean_absolute_error(y_test, predictions_test_mean)
        rmse_test = np.sqrt(mse_test)
        r2_test = r2_score(y_test, predictions_test_mean)
        print(r2_test)
        pearson_test, _ = pearsonr(y_test, predictions_test_mean)
        spearman_test, _ = spearmanr(y_test, predictions_test_mean)
        
        

        predictions.append({
            'Model': model_name,
            'Y Train pred': predictions_train,
            'Y Test actual': y_test,
            'Test prediction folds': test_predictions_folds,
            'Test Predictions Mean': predictions_test_mean,
            'Test Predictions Std': predictions_test_std,

        })

        results[model_name] = {
            'Train MSE (5 fold cv)': f"{mse_train:.4f}",
            'Train MAE (5 fold cv)': f"{mae_train:.4f}",
            'Train RMSE (5 fold cv)': f"{rmse_train:.4f}",
            'Train RMSE (5 fold cv)': f"{rmse_train:.4f}",
            'Train R2 (5 fold cv)': f"{r2_train:.4f}",
            'Train PCC (5 fold cv)': f"{pearson_train:.4f}",
            'Train SCC (5 fold cv)': f"{spearman_train:.4f}",
            'Test MSE': f"{mse_test:.4f}",
            'Test MAE': f"{mae_test:.4f}",
            'Test RMSE': f"{rmse_test:.4f}",
            'Test R2': f"{r2_test:.4f}",
            'Test Pearson Correlation': f"{pearson_test:.4f}",
            'Test Spearman Correlation': f"{spearman_test:.4f}",
        }

    results_df = pd.DataFrame(results).T
    predictions_df = pd.DataFrame(predictions)

    return results_df, predictions_df



In [37]:
X_train = train_data.drop(['ID','SMILES','Permeability'],axis=1)
y_train = train_data['Permeability']
print("X_train shape: ",X_train.shape)
print("y_train shape: ",y_train.shape)
print("XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX")

X_test = test_data.drop(['ID','SMILES','Permeability'],axis=1)
y_test = test_data['Permeability']
print("X_test shape: ",X_test.shape)
print("y_test shape: ",y_test.shape)
print("XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX")
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
X_train = pd.DataFrame(X_train_scaled, columns=X_train.columns, index=X_train.index)
X_test = pd.DataFrame(X_test_scaled, columns=X_test.columns, index=X_test.index)
models = [
    lgb.LGBMRegressor(objective='regression',metric='rmse',boosting_type='gbdt',num_leaves=31,learning_rate=0.05,random_state=101),
    DecisionTreeRegressor(random_state=101),
    RandomForestRegressor(n_jobs=-1, random_state=101),
    GradientBoostingRegressor(random_state=101),
    AdaBoostRegressor(random_state=101),
    xgb.XGBRegressor(random_state=101),
    ExtraTreesRegressor(n_jobs=-1, n_estimators=100, random_state=101),
    LinearRegression(), 
    KNeighborsRegressor(n_neighbors=3),
    SVR(),  
    MLPRegressor(random_state=101)
]
result_df, prediction_df = train_and_test_predict(models, X_train,y_train, X_test,  y_test)
result_df

X_train shape:  (140, 768)
y_train shape:  (140,)
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
X_test shape:  (36, 768)
y_test shape:  (36,)
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002131 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 29952
[LightGBM] [Info] Number of data points in the train set: 112, number of used features: 768
[LightGBM] [Info] Start training from score -5.527679
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further sp

,Train MSE (5 fold cv),Train MAE (5 fold cv),Train RMSE (5 fold cv),Train R2 (5 fold cv),Train PCC (5 fold cv),Train SCC (5 fold cv),Test MSE,Test MAE,Test RMSE,Test R2,Test Pearson Correlation,Test Spearman Correlation
LGBMRegressor,0.1211,0.2674,0.3480,0.7298,0.8544,0.8715,0.8577,0.6132,0.9261,-0.1866,0.3483,0.4375
DecisionTreeRegressor,0.2100,0.3452,0.4583,0.5315,0.7671,0.7898,0.7954,0.5853,0.8918,-0.1003,0.4034,0.4821
RandomForestRegressor,0.1065,0.2501,0.3263,0.7625,0.8737,0.8856,0.8064,0.5930,0.8980,-0.1156,0.3800,0.4680
GradientBoostingRegressor,0.1232,0.2648,0.3510,0.7252,0.8547,0.8722,0.8374,0.6082,0.9151,-0.1584,0.3719,0.4671
AdaBoostRegressor,0.1123,0.2537,0.3351,0.7495,0.8659,0.8778,0.8269,0.5937,0.9093,-0.1440,0.3655,0.4776
XGBRegressor,0.1373,0.2794,0.3705,0.6938,0.8379,0.8520,0.8165,0.5970,0.9036,-0.1295,0.3752,0.4576
ExtraTreesRegressor,0.1134,0.2521,0.3367,0.7471,0.8655,0.8765,0.8291,0.5980,0.9105,-0.1469,0.3719,0.4583
LinearRegression,1.0610,0.7548,1.0301,-1.3669,0.4844,0.5291,1.1403,0.7440,1.0678,-0.5775,0.3203,0.4615
KNeighborsRegressor,0.1206,0.2618,0.3473,0.7309,0.8564,0.8600,0.8511,0.5711,0.9226,-0.1775,0.3281,0.4126
SVR,0.1275,0.2680,0.3570,0.7156,0.8469,0.8634,0.8616,0.6052,0.9282,-0.1919,0.3424,0.4253


In [38]:
result_df

,Train MSE (5 fold cv),Train MAE (5 fold cv),Train RMSE (5 fold cv),Train R2 (5 fold cv),Train PCC (5 fold cv),Train SCC (5 fold cv),Test MSE,Test MAE,Test RMSE,Test R2,Test Pearson Correlation,Test Spearman Correlation
LGBMRegressor,0.1211,0.2674,0.3480,0.7298,0.8544,0.8715,0.8577,0.6132,0.9261,-0.1866,0.3483,0.4375
DecisionTreeRegressor,0.2100,0.3452,0.4583,0.5315,0.7671,0.7898,0.7954,0.5853,0.8918,-0.1003,0.4034,0.4821
RandomForestRegressor,0.1065,0.2501,0.3263,0.7625,0.8737,0.8856,0.8064,0.5930,0.8980,-0.1156,0.3800,0.4680
GradientBoostingRegressor,0.1232,0.2648,0.3510,0.7252,0.8547,0.8722,0.8374,0.6082,0.9151,-0.1584,0.3719,0.4671
AdaBoostRegressor,0.1123,0.2537,0.3351,0.7495,0.8659,0.8778,0.8269,0.5937,0.9093,-0.1440,0.3655,0.4776
XGBRegressor,0.1373,0.2794,0.3705,0.6938,0.8379,0.8520,0.8165,0.5970,0.9036,-0.1295,0.3752,0.4576
ExtraTreesRegressor,0.1134,0.2521,0.3367,0.7471,0.8655,0.8765,0.8291,0.5980,0.9105,-0.1469,0.3719,0.4583
LinearRegression,1.0610,0.7548,1.0301,-1.3669,0.4844,0.5291,1.1403,0.7440,1.0678,-0.5775,0.3203,0.4615
KNeighborsRegressor,0.1206,0.2618,0.3473,0.7309,0.8564,0.8600,0.8511,0.5711,0.9226,-0.1775,0.3281,0.4126
SVR,0.1275,0.2680,0.3570,0.7156,0.8469,0.8634,0.8616,0.6052,0.9282,-0.1919,0.3424,0.4253


In [39]:
prediction_df

,Model,Y Train pred,Y Test actual,Test prediction folds,Test Predictions Mean,Test Predictions Std
0,LGBMRegressor,"[-6.461972129591142, -5.712591374803092, -4.92...",0 -6.340 1 -5.760 2 -8.000 3 -6.46...,"[[-6.556433809383878, -5.939785486118237, -5.3...","[-6.529677320428773, -5.903632270581449, -5.36...","[0.10184797471022301, 0.04151551099938772, 0.0..."
1,DecisionTreeRegressor,"[-6.78, -5.85, -4.9, -5.21, -5.91, -5.32, -5.6...",0 -6.340 1 -5.760 2 -8.000 3 -6.46...,"[[-6.92, -5.87, -5.400000000000001, -6.46, -5....","[-6.548, -5.902, -5.508, -6.272, -5.444, -5.19...","[0.3451028832102103, 0.03919183588453088, 0.22..."
2,RandomForestRegressor,"[-6.7093000000000025, -5.958449999999999, -4.8...",0 -6.340 1 -5.760 2 -8.000 3 -6.46...,"[[-6.674499999999999, -5.873050000000001, -5.4...","[-6.635099999999999, -5.84768, -5.401759999999...","[0.0957920038416563, 0.049702682020190354, 0.0..."
3,GradientBoostingRegressor,"[-6.9442697017307005, -5.832083235943044, -4.8...",0 -6.340 1 -5.760 2 -8.000 3 -6.46...,"[[-6.475267594259468, -5.928201329814934, -5.4...","[-6.5576461521697365, -5.887801324122342, -5.4...","[0.1379925476508798, 0.03321466123183408, 0.03..."
4,AdaBoostRegressor,"[-6.829166666666667, -5.985227272727273, -4.73...",0 -6.340 1 -5.760 2 -8.000 3 -6.46...,"[[-6.672499999999999, -5.929242424242423, -5.3...","[-6.616155555555555, -5.8864248914981, -5.3708...","[0.09500247820029555, 0.03063347997175617, 0.0..."
5,XGBRegressor,"[-6.7287827, -5.8825006, -4.9424233, -5.429304...",0 -6.340 1 -5.760 2 -8.000 3 -6.46...,"[[-6.5853024, -5.9217157, -5.2325478, -6.24525...","[-6.6035523, -5.9263663, -5.3730116, -6.170708...","[0.15628809, 0.030330209, 0.09913564, 0.104402..."
6,ExtraTreesRegressor,"[-6.848, -5.990500000000004, -4.90180000000000...",0 -6.340 1 -5.760 2 -8.000 3 -6.46...,"[[-6.610599999999996, -5.9129499999999995, -5....","[-6.559319999999995, -5.911190000000001, -5.39...","[0.08895350246055261, 0.015197216850461678, 0...."
7,LinearRegression,"[-7.378842712424276, -5.215545399452555, -4.24...",0 -6.340 1 -5.760 2 -8.000 3 -6.46...,"[[-7.622220042423139, -6.040349668541483, -5.4...","[-7.6873916258642625, -6.061553752227865, -5.5...","[0.41957654040530673, 0.09335707082113734, 0.1..."
8,KNeighborsRegressor,"[-6.796666666666667, -6.289999999999999, -5.07...",0 -6.340 1 -5.760 2 -8.000 3 -6.46...,"[[-6.260000000000001, -5.983333333333333, -5.2...","[-6.398000000000001, -5.924666666666667, -5.32...","[0.12235830807736585, 0.05153639490690065, 0.0..."
9,SVR,"[-6.294801464757613, -5.964836217434125, -4.97...",0 -6.340 1 -5.760 2 -8.000 3 -6.46...,"[[-6.264768394585466, -5.848149182899448, -5.3...","[-6.358650855395167, -5.844271482290285, -5.35...","[0.07063454922363697, 0.036321021590889846, 0...."


In [40]:
result_df.to_csv('/kaggle/working/Results_ChemBERTa-zinc-base-v1_fine_tuned_embeddings_rrck.csv')
prediction_df.to_csv('/kaggle/working/Prediction_data_ChemBERTa-zinc-base-v1_fine_tuned_embeddings_rrck.csv')

In [41]:
result_df

,Train MSE (5 fold cv),Train MAE (5 fold cv),Train RMSE (5 fold cv),Train R2 (5 fold cv),Train PCC (5 fold cv),Train SCC (5 fold cv),Test MSE,Test MAE,Test RMSE,Test R2,Test Pearson Correlation,Test Spearman Correlation
LGBMRegressor,0.1211,0.2674,0.3480,0.7298,0.8544,0.8715,0.8577,0.6132,0.9261,-0.1866,0.3483,0.4375
DecisionTreeRegressor,0.2100,0.3452,0.4583,0.5315,0.7671,0.7898,0.7954,0.5853,0.8918,-0.1003,0.4034,0.4821
RandomForestRegressor,0.1065,0.2501,0.3263,0.7625,0.8737,0.8856,0.8064,0.5930,0.8980,-0.1156,0.3800,0.4680
GradientBoostingRegressor,0.1232,0.2648,0.3510,0.7252,0.8547,0.8722,0.8374,0.6082,0.9151,-0.1584,0.3719,0.4671
AdaBoostRegressor,0.1123,0.2537,0.3351,0.7495,0.8659,0.8778,0.8269,0.5937,0.9093,-0.1440,0.3655,0.4776
XGBRegressor,0.1373,0.2794,0.3705,0.6938,0.8379,0.8520,0.8165,0.5970,0.9036,-0.1295,0.3752,0.4576
ExtraTreesRegressor,0.1134,0.2521,0.3367,0.7471,0.8655,0.8765,0.8291,0.5980,0.9105,-0.1469,0.3719,0.4583
LinearRegression,1.0610,0.7548,1.0301,-1.3669,0.4844,0.5291,1.1403,0.7440,1.0678,-0.5775,0.3203,0.4615
KNeighborsRegressor,0.1206,0.2618,0.3473,0.7309,0.8564,0.8600,0.8511,0.5711,0.9226,-0.1775,0.3281,0.4126
SVR,0.1275,0.2680,0.3570,0.7156,0.8469,0.8634,0.8616,0.6052,0.9282,-0.1919,0.3424,0.4253


In [42]:
prediction_df

,Model,Y Train pred,Y Test actual,Test prediction folds,Test Predictions Mean,Test Predictions Std
0,LGBMRegressor,"[-6.461972129591142, -5.712591374803092, -4.92...",0 -6.340 1 -5.760 2 -8.000 3 -6.46...,"[[-6.556433809383878, -5.939785486118237, -5.3...","[-6.529677320428773, -5.903632270581449, -5.36...","[0.10184797471022301, 0.04151551099938772, 0.0..."
1,DecisionTreeRegressor,"[-6.78, -5.85, -4.9, -5.21, -5.91, -5.32, -5.6...",0 -6.340 1 -5.760 2 -8.000 3 -6.46...,"[[-6.92, -5.87, -5.400000000000001, -6.46, -5....","[-6.548, -5.902, -5.508, -6.272, -5.444, -5.19...","[0.3451028832102103, 0.03919183588453088, 0.22..."
2,RandomForestRegressor,"[-6.7093000000000025, -5.958449999999999, -4.8...",0 -6.340 1 -5.760 2 -8.000 3 -6.46...,"[[-6.674499999999999, -5.873050000000001, -5.4...","[-6.635099999999999, -5.84768, -5.401759999999...","[0.0957920038416563, 0.049702682020190354, 0.0..."
3,GradientBoostingRegressor,"[-6.9442697017307005, -5.832083235943044, -4.8...",0 -6.340 1 -5.760 2 -8.000 3 -6.46...,"[[-6.475267594259468, -5.928201329814934, -5.4...","[-6.5576461521697365, -5.887801324122342, -5.4...","[0.1379925476508798, 0.03321466123183408, 0.03..."
4,AdaBoostRegressor,"[-6.829166666666667, -5.985227272727273, -4.73...",0 -6.340 1 -5.760 2 -8.000 3 -6.46...,"[[-6.672499999999999, -5.929242424242423, -5.3...","[-6.616155555555555, -5.8864248914981, -5.3708...","[0.09500247820029555, 0.03063347997175617, 0.0..."
5,XGBRegressor,"[-6.7287827, -5.8825006, -4.9424233, -5.429304...",0 -6.340 1 -5.760 2 -8.000 3 -6.46...,"[[-6.5853024, -5.9217157, -5.2325478, -6.24525...","[-6.6035523, -5.9263663, -5.3730116, -6.170708...","[0.15628809, 0.030330209, 0.09913564, 0.104402..."
6,ExtraTreesRegressor,"[-6.848, -5.990500000000004, -4.90180000000000...",0 -6.340 1 -5.760 2 -8.000 3 -6.46...,"[[-6.610599999999996, -5.9129499999999995, -5....","[-6.559319999999995, -5.911190000000001, -5.39...","[0.08895350246055261, 0.015197216850461678, 0...."
7,LinearRegression,"[-7.378842712424276, -5.215545399452555, -4.24...",0 -6.340 1 -5.760 2 -8.000 3 -6.46...,"[[-7.622220042423139, -6.040349668541483, -5.4...","[-7.6873916258642625, -6.061553752227865, -5.5...","[0.41957654040530673, 0.09335707082113734, 0.1..."
8,KNeighborsRegressor,"[-6.796666666666667, -6.289999999999999, -5.07...",0 -6.340 1 -5.760 2 -8.000 3 -6.46...,"[[-6.260000000000001, -5.983333333333333, -5.2...","[-6.398000000000001, -5.924666666666667, -5.32...","[0.12235830807736585, 0.05153639490690065, 0.0..."
9,SVR,"[-6.294801464757613, -5.964836217434125, -4.97...",0 -6.340 1 -5.760 2 -8.000 3 -6.46...,"[[-6.264768394585466, -5.848149182899448, -5.3...","[-6.358650855395167, -5.844271482290285, -5.35...","[0.07063454922363697, 0.036321021590889846, 0...."
